# Analyze Data

In [ ]:
library(dplyr, warn.conflicts=FALSE)
library(ggplot2)
library(gtools)
library(MASS)
library(xtable)
library(bit64)
library(scales)
library(table1)
library(patchwork)

In [ ]:
here::i_am("analysis.ipynb")

In [ ]:
source(here::here('R', 'plot.R'))
source(here::here('R', 'io.R'))
source(here::here('R', 'check_hashsum.R'))
source(here::here('R', 'inverse_probability_weighting.R'))

## Load data

In [ ]:
data_dir <- here::here('data')
results_dir <- here::here('results')

In [ ]:
df_file <- file.path(data_dir, 'feature_table.parquet')

In [ ]:
df <- read_parquet_table(df_file)

## Analyze

In [ ]:
dim(df)
table(as.numeric(!is.na(df$co_cancer_first_age_in_years)))

In [ ]:
df2 <- 
  df %>%
  dplyr::mutate(
    co_diabetes_first_age_in_years = as.numeric(!is.na(co_diabetes_first_age_in_years)),
    co_hypertension_first_age_in_years = as.numeric(!is.na(co_hypertension_first_age_in_years)),
    co_ckd_first_age_in_years = as.numeric(!is.na(co_ckd_first_age_in_years)),
    co_liver_disease_first_age_in_years = as.numeric(!is.na(co_liver_disease_first_age_in_years)),
    co_cancer_first_age_in_years = as.numeric(!is.na(co_cancer_first_age_in_years)),
      co_immunocompromised_first_age_in_years = as.numeric(!is.na(co_immunocompromised_first_age_in_years)),
      co_hiv_first_age_in_years = as.numeric(!is.na(co_hiv_first_age_in_years)),
    vital_last_oxygen_value = as.numeric(vital_last_oxygen_value)
  ) %>%
  as.data.frame()

# the warning is from vital_last_oxygen_value
# it is ignorable

In [ ]:
#head(df2$co_liver_disease_first_age_in_years)
dim(df2)
dim(df2[df2$encounter_class %in% "inpatient encounter",])
length(which(df2$encounter_class %in% "inpatient encounter"))


In [ ]:
# --- make race a factor and relevel to white as baseline
df2$race.factor <- relevel(factor(df2$race), "White")

# --- indicator for AA/white
df2$aa.black <- as.numeric(df2$race%in%"Black or African American")
df2$white <- as.numeric(df2$race%in%"White")


# --- make ages into years
df2$age.years <- (df2$matched_age) / 60 / 60 / 24 / 365

# --- mean centered age
df2$age.years.centered <- (df2$age.years-mean(df2$age.years))#/sd(df2$age.years)

# --- comorbidities into yes/no
df2$copd.ever <- !is.na(df2$co_chronic_pulmonary_disease_first_age_in_years)

df2$ckd.ever <- !is.na(df2$co_ckd_first_age)

df2$liver.ever <- !is.na(df2$co_liver_disease_first_age)

# --- remdesivir into yes/no
df2$remdesivir.yes <- as.numeric(!is.na(df2$med_remdesivir_age_group))

# --- azithromycin into yes/no
df2$azithromycin.yes <- as.numeric(!is.na(df2$med_azithromycin_age_group))



In [ ]:
inpatient <- 
  df2 %>% 
  dplyr::filter(encounter_class %in% c('inpatient encounter', 'observation encounter'))

## Table 1

In [ ]:
for_t1 <-
  inpatient |> 
  dplyr::rename(remdesivir = remdesivir.yes) |>
  dplyr::mutate(
    remdesivir = dplyr::if_else(remdesivir == 1, TRUE, FALSE),
    matched_age_group = 
      dplyr::if_else(matched_age_group == '[5,10)', '[05,10)', matched_age_group),
    dplyr::across(
      c(
        co_cancer_first_age, 
        co_diabetes_first_age,
        co_hypertension_first_age, 
        co_ckd_first_age,
        co_liver_disease_first_age, 
        co_immunocompromised_first_age
      ), 
      ~ dplyr::if_else(is.na(.x), FALSE, TRUE)
    ),
    population = stringr::str_to_sentence(population),
    population = factor(population, levels = c('Wild', 'Dec2020', 'Delta', 'Omicron'))
  ) |>
  dplyr::rename_with(
    .cols = 
      c(
        co_cancer_first_age, 
        co_diabetes_first_age,
        co_hypertension_first_age, 
        co_ckd_first_age, 
        co_liver_disease_first_age, 
        co_immunocompromised_first_age
      ), 
    .fn = ~ stringr::str_remove(., '_first_age')
  ) |>
  dplyr::select(
    population,
    remdesivir,
    sex,
    race, 
    ethnicity, 
    matched_age_group, 
    co_cancer,
    co_ckd, 
    co_diabetes,
    co_hypertension, 
    co_liver_disease, 
    co_immunocompromised
  )

In [ ]:
# give nice labels to everything

label(for_t1$population) <- 'COVID Wave'
label(for_t1$remdesivir) <- 'Treated with Remdesivir'
label(for_t1$sex) <- 'Sex'
label(for_t1$race) <- 'Race'
label(for_t1$ethnicity) <- 'Ethnicity'
label(for_t1$matched_age_group) <- 'Age bracket'
label(for_t1$co_cancer) <- 'Cancer'
label(for_t1$co_ckd) <- 'Chronic kidney disease (CKD)'
label(for_t1$co_diabetes) <- 'Diabetes'
label(for_t1$co_hypertension) <- 'Hypertension'
label(for_t1$co_liver_disease) <- 'Liver disease'
label(for_t1$co_immunocompromised) <- 'Immunocompromised'

In [ ]:
# make a table

tab1 <- 
  table1(
    ~ sex +
      race + 
      ethnicity + 
      matched_age_group + 
      remdesivir + 
      co_cancer + 
      co_ckd + 
      co_diabetes +
      co_hypertension + 
      co_liver_disease + 
      co_immunocompromised |
      population, 
    data = for_t1, 
    overall = 'Overall'
  )

In [ ]:
as.data.frame(tab1)
write.csv(
  as.data.frame(tab1, make.names = FALSE), 
  file = here::here('results', 'table_one_inpatient.csv'), 
  row.names = FALSE
)

In [ ]:
xtab_1 <- xtable::xtable(as.data.frame(tab1, make.names = FALSE))
print.xtable(
  xtab_1, 
  type = 'latex', 
  file = here::here('results', 'table_1_inpatient.tex'), 
  include.rownames = FALSE
)

## histogram of age distributions

In [ ]:
for_age_hist <- 
  inpatient |>    
  dplyr::filter(race %in% c("White", "Black or African American")) |>
  dplyr::mutate(
    matched_age_group = 
      dplyr::if_else(matched_age_group == '[5,10)', '[05,10)', matched_age_group),
    population = stringr::str_to_sentence(population),
    population = factor(population, levels = c('Wild', 'Dec2020', 'Delta', 'Omicron'))
  )

age_hist <- 
  for_age_hist |>
  dplyr::group_by(race, matched_age_group) |>
  dplyr::count(name = 'count') |>
  dplyr::group_by(race) |>
  dplyr::mutate(freq = count / sum(count)) |>
  ggplot(aes(x = matched_age_group, y = freq)) +
  geom_bar(stat = 'identity') +
  facet_wrap(~ race, ncol = 1) +
  theme_truveta() +
  theme(
    axis.text.x = element_text(colour = 'black', angle = 45, vjust = 1.5, hjust = 1)
  ) +
  labs(x = 'Age bin (years)', y = 'Frequency')

age_hist

ggsave(
  plot = age_hist, 
  filename = file.path(results_dir, 'age_distribution_inpatient.pdf'),
  width = 6,
  height = 6
)

In [ ]:
for_age_wave_hist <- 
  for_age_hist |>
  dplyr::mutate(
    population = factor(population, levels = c('Wild', 'Dec2020', 'Delta', 'Omicron'))
  ) |>
  dplyr::group_by(population, race, matched_age_group) |>
  dplyr::count(name = 'count') |> 
  dplyr::group_by(population, race) |>
  dplyr::mutate(freq = count / sum(count))

In [ ]:
age_wave_race_hist <- 
  for_age_wave_hist |>
  ggplot(aes(x = matched_age_group, y = freq)) +
  geom_bar(stat = 'identity') +
  facet_grid(race ~ population, scales = 'free_y', switch = 'y') +
  theme_truveta() +
  theme(
    axis.text.x = element_text(size = 7, angle = 90, vjust = 0.5, hjust=1),
    strip.placement = 'outside'
  ) +
  labs(x = 'Age bin (years)', y = 'Frequency')

age_wave_race_hist

ggsave(
  plot = age_wave_race_hist, 
  filename = file.path(results_dir, 'age_race_wave_distribution_inpatient.pdf'),
  width = 10,
  height = 5
)

### remdesivir just inpatient w/ small list of predictors

In [ ]:
pred <- c('race.factor', 'sex', 'age.years')

## logistic regression

In [ ]:
logistic_med_pop <- function(df, which_med, which_population, predictors) {
    
  data <- 
    df %>%
    dplyr::filter(
      population == which_population,
      !is.na(race)
    ) %>%
    dplyr::mutate(y = as.numeric(!is.na({{ which_med }}))) %>%
    dplyr::select(y, dplyr::all_of({{ predictors }}))
  
  mod <- glm(y ~ ., data = na.omit(data), family = 'binomial')
  #mod.step <- step(mod)

  mod_summary <- summary(mod)

  mod_coef <- cbind(exp(coef(mod)), exp(confint(mod)))
  mod_coef_xtable <- xtable::xtable(mod_coef)

  #mod_step_coef <- cbind(exp(coef(mod.step)), exp(confint(mod.step)))
  
  out <- 
    list(
      mod = mod, 
      summary = mod_summary, 
      coef = mod_coef, 
      coef_xtable = mod_coef_xtable#,
      #step_coef = mod_step_coef
    )

  out
}

In [ ]:
pred <- c('race.factor', 'sex', 'age.years')

In [ ]:
wild_remdesivir_inpatient <- logistic_med_pop(inpatient, med_remdesivir_age_group, 'wild', pred)

write.csv(
  wild_remdesivir_inpatient$coef, 
  file.path(results_dir, 'wild_remdesivir_coef_inpatient.csv')
)

print.xtable(
  wild_remdesivir_inpatient$coef_xtable, 
  file = file.path(results_dir, 'wild_remdesivir_coef_inpatient.tex')
)

print(wild_remdesivir_inpatient$coef_xtable)

In [ ]:
dec2020_remdesivir_inpatient <- logistic_med_pop(inpatient, med_remdesivir_age_group, 'dec2020', pred)

write.csv(
  dec2020_remdesivir_inpatient$coef, 
  file.path(results_dir, 'dec2020_remdesivir_coef_inpatient.csv')
)

print.xtable(
  dec2020_remdesivir_inpatient$coef_xtable, 
  file = file.path(results_dir, 'dec2020_remdesivir_coef_inpatient.tex')
)

print(dec2020_remdesivir_inpatient$coef_xtable)

In [ ]:
delta_remdesivir_inpatient <- logistic_med_pop(inpatient, med_remdesivir_age_group, 'delta', pred)

write.csv(
  delta_remdesivir_inpatient$coef, 
  file.path(results_dir, 'delta_remdesivir_coef_inpatient.csv')
)

print.xtable(
  delta_remdesivir_inpatient$coef_xtable, 
  file = file.path(results_dir, 'delta_remdesivir_coef_inpatient.tex')
)

print(delta_remdesivir_inpatient$coef_xtable)

In [ ]:
omicron_remdesivir_inpatient <- logistic_med_pop(inpatient, med_remdesivir_age_group, 'omicron', pred)

write.csv(
  omicron_remdesivir_inpatient$coef, 
  file.path(results_dir, 'omicron_remdesivir_coef_inpatient.csv')
)

print.xtable(
  omicron_remdesivir_inpatient$coef_xtable, 
  file = file.path(results_dir, 'omicron_remdesivir_coef_inpatient.tex')
)

print(omicron_remdesivir_inpatient$coef_xtable)

In [ ]:
# tab.omicron == omicron_remdesivir_large$coef
tt <- 
  cbind(#wild_azithromycin_large$coef,
        omicron_remdesivir_inpatient$coef,
        delta_remdesivir_inpatient$coef,
        omicron_remdesivir_inpatient$coef)


# base plotting
pdf(file.path(results_dir, "logistic_regressions_inpatient.pdf"), 4,4)
par(oma=c(2,2,0,0), tck = 0.01, cex.axis=0.65, mgp = c(2,.5,0), mar = c(0.5,0.5,0.5,0.5))
x <- barplot(tt[4,c(1,4,7)], ylim=c(0,1.25))
abline(h=1, lty=2)
segments(rep(x,3), tt[4,c(2,5,8)], rep(x,3), tt[4,c(3,6,9)])
text(x, -.05, c("Dec. 2020","Delta","Omicron"), srt=45, xpd=NA, cex=1.25, pos=2)
mtext("OR remdesivir treatment",2, line=2)
dev.off()


### Remdesivir inpatient + additional predictors

In [ ]:
pred_rem_large <- c("race.factor",
                    "sex",
                    "age.years",
                    "co_diabetes_first_age_in_years",
                    "co_hypertension_first_age_in_years",
                    "co_ckd_first_age_in_years",
                    "co_liver_disease_first_age_in_years",
                    "co_immunocompromised_first_age_in_years",
                   "co_cancer_first_age_in_years")



In [ ]:
summary_race_age_inpatient <- 
  inpatient |> 
  dplyr::filter(race %in% c('White', 'Black or African American')) |> 
  dplyr::group_by(race) |>
  dplyr::summarize(
    age_median = median(age.years),
    iqr_age = IQR(age.years),
    age_q25 = quantile(age.years, 0.25),
    age_q75 = quantile(age.years, 0.75)
  )

write.csv(summary_race_age_inpatient, file = file.path(results_dir, 'summary_race_age_inpatient.csv'))

In [ ]:
# ---------- calculate ecdfs
sample1 <- inpatient$age.years[inpatient$aa.black==1]
sample2 <- inpatient$age.years[inpatient$white==1]
group <- c(rep("AA.or.black", length(sample1)), rep("White", length(sample2)))
dat <- data.frame(KSD = c(sample1, sample2), group = group)
# create ECDF of data
cdf1 <- ecdf(sample1)
cdf2 <- ecdf(sample2)

# find min and max statistics to draw line between points of greatest distance
minMax <- seq(min(sample1, sample2), max(sample1, sample2), length.out=length(sample1)) 
x0 <- minMax[which( abs(cdf1(minMax) - cdf2(minMax)) == max(abs(cdf1(minMax) - cdf2(minMax))) )] 
y0 <- cdf1(x0)
y1 <- cdf2(x0)

In [ ]:
# ---- plots
plot(cdf1, col="black") 
lines(cdf2, col="red") 

points(c(x0, x0), c(y0, y1), pch=16, col="red") 
segments(x0, y0, x0, y1, col="red", lty="dotted") 


# ---- do a KS test b/w groups
ks_results <- ks.test(sample1, sample2, alternative = "greater")


# ---- compare medians
round(quantile(sample1, c(0.5, 0.25, 0.75)), 1)
round(quantile(sample2, c(0.5, 0.25, 0.75)), 1)

In [ ]:
ks_tidy <- broom::tidy(ks_results)

write.csv(ks_tidy, file = file.path(results_dir, 'ks_race_results_inpatient.csv'))

In [ ]:
# colnames(inpatient)

### race + population comorbidity counts + fractions

i hate having to roll my own solution here, but couldn't figure out gtsummary solution quick enough

In [ ]:
# prepare to summarize race X population X comorbidity summary
race_wave_comorbid_prep <- 
  inpatient |> 
  dplyr::rename(
    cancer = co_cancer_first_age,
    ckd = co_ckd_first_age,
    diabetes = co_diabetes_first_age,
    hypertension = co_hypertension_first_age,
    liver_disease = co_liver_disease_first_age,
    immunocompromised = co_immunocompromised_first_age
  ) |>
  dplyr::select(race, population, cancer, ckd, diabetes, hypertension, liver_disease, immunocompromised) |>
  dplyr::mutate(
    dplyr::across(
      cancer:immunocompromised, 
      ~ dplyr::if_else(is.na(.x), FALSE, TRUE)
    ),
    population = factor(population, levels = c('wild', 'dec2020', 'delta', 'omicron'))
  ) |>
  dplyr::filter(race %in% c("White", "Black or African American"))

In [ ]:
# first, ignore population and do overall

race_comorbid_fraction <- 
  race_wave_comorbid_prep |>
  group_by(race) |> 
  dplyr::summarize(
    dplyr::across(
      cancer:immunocompromised, 
      ~ round(sum(.x) / n(), 4)
    ),
    .groups = 'drop'
  ) |> 
  tidyr::pivot_longer(
    cols = cancer:immunocompromised, 
    names_to = 'comorbidity', 
    values_to = 'fraction'
  ) |>
  dplyr::mutate(
    comorbidity = stringr::str_replace(comorbidity, '_', ' '),
    comorbidity = stringr::str_to_sentence(comorbidity),
    comorbidity = dplyr::if_else(comorbidity == 'Ckd', 'CKD', comorbidity)
  )

race_comorbid_count <- 
  race_wave_comorbid_prep |>
  group_by(race) |> 
  dplyr::summarize(
    dplyr::across(
      cancer:immunocompromised, 
      ~ sum(.x)
    ),
    .groups = 'drop'
  ) |> 
  tidyr::pivot_longer(
    cols = cancer:immunocompromised, 
    names_to = 'comorbidity', 
    values_to = 'count'
  ) |>
  dplyr::mutate(
    comorbidity = stringr::str_replace(comorbidity, '_', ' '),
    comorbidity = stringr::str_to_sentence(comorbidity),
    comorbidity = dplyr::if_else(comorbidity == 'Ckd', 'CKD', comorbidity)
  )

In [ ]:
# combine count and fraction into single table
race_comorbid_summary <- 
  inner_join(
    race_comorbid_fraction, 
    race_comorbid_count,
    by = c('race' = 'race', 'comorbidity' = 'comorbidity')
  ) |>
  dplyr::rename(
    Race = race,
    Comorbidity = comorbidity
  ) |>
  dplyr::mutate(
    Overall = paste0(count, ' (', fraction * 100, '%)')
  ) |>
  dplyr::select(-(count:fraction))

race_comorbid_summary

write.csv(
  race_comorbid_summary, 
  file = file.path(results_dir, 'race_comorbid_summary_inpatient.csv')
)

print.xtable(
  xtable::xtable(race_comorbid_summary, make.names = FALSE),
  type = 'latex', 
  file = file.path(results_dir, 'race_comorbid_summary_inpatient.tex'), 
  include.rownames = FALSE
)

In [ ]:
# fraction with comorbidity by race and population
race_wave_comorbid_fraction <-
  race_wave_comorbid_prep |>
  dplyr::group_by(race, population) |>
  dplyr::summarize(
    dplyr::across(
      cancer:immunocompromised, 
      ~ round(sum(.x) / n(), 4)
    ),
    .groups = 'drop'
  ) 

race_wave_comorbid_fraction_summary <- 
  race_wave_comorbid_fraction |> 
  tidyr::pivot_longer(
    cols = cancer:immunocompromised, 
    names_to = 'comorbidity', 
    values_to = 'fraction'
  ) |>
  tidyr::pivot_wider(names_from = population, values_from = fraction) |>
  dplyr::mutate(
    comorbidity = stringr::str_replace(comorbidity, '_', ' '),
    comorbidity = stringr::str_to_sentence(comorbidity),
    comorbidity = dplyr::if_else(comorbidity == 'Ckd', 'CKD', comorbidity)
  )

# count with comorbidity by race and population
race_wave_comorbid_count <-
  race_wave_comorbid_prep |>
  dplyr::group_by(race, population) |>
  dplyr::summarize(
    dplyr::across(
      cancer:immunocompromised, 
      ~ sum(.x)
    ),
    .groups = 'drop'
  ) 

race_wave_comorbid_count_summary <- 
  race_wave_comorbid_count |> 
  tidyr::pivot_longer(
    cols = cancer:immunocompromised, 
    names_to = 'comorbidity', 
    values_to = 'count'
  ) |>
  tidyr::pivot_wider(names_from = population, values_from = count) |>
  dplyr::mutate(
    comorbidity = stringr::str_replace(comorbidity, '_', ' '),
    comorbidity = stringr::str_to_sentence(comorbidity),
    comorbidity = dplyr::if_else(comorbidity == 'Ckd', 'CKD', comorbidity)
  )

In [ ]:
# combine count and fraction into single table
race_wave_comorbid_summary <- 
  inner_join(
    race_wave_comorbid_count_summary, 
    race_wave_comorbid_fraction_summary, 
    by = c('race' = 'race', 'comorbidity' = 'comorbidity')
  ) |>
  dplyr::rename(
    Race = race,
    Comorbidity = comorbidity
  ) |>
  dplyr::mutate(
    Wild = paste0(wild.x, ' (', wild.y * 100, '%)'),
    `Dec. 2020` = paste0(dec2020.x, ' (', dec2020.y * 100, '%)'),
    Delta = paste0(delta.x, ' (', delta.y * 100, '%)'),
    Omicron = paste0(omicron.x, ' (', omicron.y * 100, '%)')
  ) |>
  dplyr::select(-(wild.x:omicron.y))

race_wave_comorbid_summary

write.csv(
  race_wave_comorbid_summary, 
  file = file.path(results_dir, 'race_wave_comorbid_summary_inpatient.csv')
)

print.xtable(
  xtable::xtable(race_wave_comorbid_summary, make.names = FALSE),
  type = 'latex', 
  file = file.path(results_dir, 'race_wave_comorbid_summary_inpatient.tex'), 
  include.rownames = FALSE
)

In [ ]:
table_2 <- 
  dplyr::inner_join(
    race_wave_comorbid_summary, 
    race_comorbid_summary, 
    by = c('Race' = 'Race', 'Comorbidity' = 'Comorbidity')
  ) |> 
  dplyr::arrange(Comorbidity)

table_2

write.csv(
  table_2, 
  file = file.path(results_dir, 'table_2_inpatient.csv')
)

print.xtable(
  xtable::xtable(table_2, make.names = FALSE),
  type = 'latex', 
  file = file.path(results_dir, 'table_2_inpatient.tex'), 
  include.rownames = FALSE
)

In [ ]:
# ---- some summary plots


# ----- comorbidities by AA/white
summary_table_cov <- 
  c(
    "sex",
    "co_diabetes_first_age_in_years",
    "co_hypertension_first_age_in_years",
    "co_ckd_first_age_in_years",
    "co_liver_disease_first_age_in_years",
    "co_immunocompromised_first_age_in_years",
    "co_cancer_first_age_in_years"
  )


com.tab.aa.black <- c()
for (i in 1:length(summary_table_cov)){
    com.tab.aa.black <- rbind(com.tab.aa.black, table(inpatient$aa.black, inpatient[,summary_table_cov[i]])[2,])
}
aa.black.tab <- paste0(round(com.tab.aa.black[,2]/rowSums(com.tab.aa.black)*100,1),"%")
names(aa.black.tab) <- summary_table_cov
aa.black.tab

com.tab.white <- c()
for (i in 1:length(summary_table_cov)){
    com.tab.white <- rbind(com.tab.white, table(inpatient$white, inpatient[,summary_table_cov[i]])[2,])
}
white.tab <- paste0(round(com.tab.white[,2]/rowSums(com.tab.white)*100,1),"%")
names(white.tab) <- summary_table_cov
white.tab

cbind(aa.black.tab, white.tab)

In [ ]:
# ----- now do some t-tests b/w groups
t.tab.white <- com.tab.white
t.tab.aa.black <- com.tab.aa.black
t.tab.white[,1] <- rowSums(com.tab.white)
t.tab.aa.black[,1] <- rowSums(com.tab.aa.black)


t.tab <- c()
for (i in 1:length(summary_table_cov)){
    t.tab <- c(t.tab, t.test(t.tab.aa.black[i,], t.tab.white[i,])$p.value)
}


comorbid.table <- cbind(paste0(round(t.tab.aa.black[,2]/t.tab.aa.black[,1]*100,1),"%"), white.tab, round(t.tab, 3))
colnames(comorbid.table) <- c("AA and black", "White", "p value")
rownames(comorbid.table) <- gsub("_"," ",gsub("co_","",gsub("_first_age_in_years","",summary_table_cov)))
comorbid.table


print.xtable(
  comorbid.table, 
  file = file.path(results_dir, 'comorbid_summary_table_inpatient.tex')
)

In [ ]:
wild_remdesivir_large <- logistic_med_pop(inpatient, med_remdesivir_age_group, 'wild', pred_rem_large)

write.csv(
  wild_remdesivir_large$coef, 
  file.path(results_dir, 'wild_remdesivir_coef_inpatient_large_inpatient.csv')
)

print.xtable(
  wild_remdesivir_large$coef_xtable, 
  file = file.path(results_dir, 'wild_remdesivir_coef_inpatient_large_inpatient.tex')
)

wild_remdesivir_large$coef

In [ ]:
dec2020_remdesivir_large <- logistic_med_pop(inpatient, med_remdesivir_age_group, 'dec2020', pred_rem_large)

write.csv(
  dec2020_remdesivir_large$coef, 
  file.path(results_dir, 'dec2020_remdesivir_coef_inpatient_large_inpatient.csv')
)

print.xtable(
  dec2020_remdesivir_large$coef_xtable, 
  file = file.path(results_dir, 'dec2020_remdesivir_coef_inpatient_large_inpatient.tex')
)

print(dec2020_remdesivir_large$coef_xtable)

In [ ]:
delta_remdesivir_large <- logistic_med_pop(inpatient, med_remdesivir_age_group, 'delta', pred_rem_large)

write.csv(
  delta_remdesivir_large$coef, 
  file.path(results_dir, 'delta_remdesivir_coef_inpatient_large_inpatient.csv')
)

print.xtable(
  delta_remdesivir_large$coef_xtable, 
  file = file.path(results_dir, 'delta_remdesivir_coef_inpatient_large_inpatient.tex')
)

print(delta_remdesivir_large$coef_xtable)

In [ ]:
omicron_remdesivir_large <- logistic_med_pop(inpatient, med_remdesivir_age_group, 'omicron', pred_rem_large)

write.csv(
  omicron_remdesivir_large$coef,
  file.path(results_dir, 'omicron_remdesivir_coef_inpatient_large_inpatient.csv')
)

print.xtable(
  omicron_remdesivir_large$coef_xtable, 
  file = file.path(results_dir, 'omicron_remdesivir_coef_inpatient_large_inpatient.tex')
)

print(omicron_remdesivir_large$coef_xtable)

In [ ]:
coef_list <- 
  list(
    #wild = wild_remdesivir_large$coef,
    dec2020 = dec2020_remdesivir_large$coef, 
    delta = delta_remdesivir_large$coef, 
    omicron = omicron_remdesivir_large$coef
  )

coef_log_df <- 
  Map(\(x) tibble::rownames_to_column(as.data.frame(x)), coef_list) |> 
  bind_rows(.id = 'population') |>
  janitor::clean_names()

black_log_or <- 
  coef_log_df |> 
  dplyr::filter(rowname %in% c('race.factorBlack or African American')) |>
  dplyr::mutate(
    population = stringr::str_to_title(population),
    #population = factor(population, levels = c("Wild", "Dec2020", "Delta", "Omicron"))
    population = factor(population, levels = c("Dec2020", "Delta", "Omicron"))
  )

In [ ]:
black_log_or_tab <- 
  black_log_or |> 
  dplyr::mutate(
    rowname = 'Black or African American',
    across(v1:x97_5_percent, ~ round(.x, 4))
  ) |>
  dplyr::rename(
    Estimate = v1,
    `2.5%` = x2_5_percent,
    `97.5%` = x97_5_percent
  ) |>
  dplyr::rename(
    Wave = population,
    Population = rowname
  )

print.xtable(
  xtable::xtable(black_log_or_tab, make.names = FALSE),
  type = 'latex', 
  file = file.path(results_dir, 'adjusted_logistic_or_inpatient.tex'), 
  include.rownames = FALSE
)

write.csv(
  black_log_or_tab, 
  file = file.path(results_dir, 'adjusted_logistic_or_inpatient.csv')
)

In [ ]:
black_log_or_graph <- 
  ggplot(black_log_or, aes(x = population, y = v1)) +
  geom_hline(yintercept = 1, linetype = 'dashed') +
  geom_pointrange(mapping = aes(ymin = x2_5_percent, ymax = x97_5_percent)) +
  labs(x = '', y = stringr::str_wrap('Logistic regression OR remdesivir treatment', 35)) + 
  theme_bw() +
  theme(axis.text.x = element_text(colour = 'black', angle = 45, vjust = 0.5, hjust = 1))

black_log_or_graph

In [ ]:
black_log_or_graph_short <- black_log_or_graph
black_log_or_graph <- black_log_or_graph + ylim(0, NA)


ggsave(
  plot = black_log_or_graph, 
  filename = here::here('results', 'adjusted_logistic_regressions_inpatient.pdf'), 
  width = 4, 
  height = 4
)
ggsave(
  plot = black_log_or_graph, 
  filename = here::here('results', 'adjusted_logistic_regressions_inpatient.png'), 
  width = 4, 
  height = 4
)
ggsave(
  plot = black_log_or_graph, 
  filename = here::here('results', 'adjusted_logistic_regressions_inpatient.svg'), 
  width = 4, 
  height = 4
)

ggsave(
  plot = black_log_or_graph_short, 
  filename = here::here('results', 'adjusted_logistic_regressions_short_y_axis_inpatient.pdf'), 
  width = 4, 
  height = 4
)
ggsave(
  plot = black_log_or_graph_short, 
  filename = here::here('results', 'adjusted_logistic_regressions_short_y_axis_inpatient.png'), 
  width = 4, 
  height = 4
)
ggsave(
  plot = black_log_or_graph_short, 
  filename = here::here('results', 'adjusted_logistic_regressions_short_y_axis_inpatient.svg'), 
  width = 4, 
  height = 4
)

## Count regression

In [ ]:
pois_inpatient <- 
  inpatient %>%  # defined earlier
  dplyr::mutate(
    los = (encounter_end_age - encounter_start_age) / 60 / 60 / 24,
    los = dplyr::if_else(los <= 0, NA_integer64_, los),
    los = round(los, 0)
  ) %>%
  dplyr::filter(!is.na(los)) %>%
  dplyr::mutate(los = as.integer(los))  # int64 causes problems with glm


pred_rem_large <- c("race.factor",
                    "sex",
                    "age.years",
                    "co_diabetes_first_age_in_years",
                    "co_hypertension_first_age_in_years",
                    "co_ckd_first_age_in_years",
                    "co_liver_disease_first_age_in_years",
                    "co_immunocompromised_first_age_in_years",
                   "co_cancer_first_age_in_years")


pred_pois <- c("race.factor",
               "sex",
               "age.years", 
               "co_diabetes_first_age_in_years", 
               "co_hypertension_first_age_in_years",
               "co_ckd_first_age_in_years",
               "co_liver_disease_first_age_in_years",
               "co_immunocompromised_first_age_in_years",
               "co_cancer_first_age_in_years"
              )



In [ ]:
df_reg <- 
  pois_inpatient %>%
  dplyr::filter(!is.na(race)) %>%
  dplyr::select(los, dplyr::all_of(pred_pois))

overall_mod <- glm(los ~ ., data = df_reg, family = 'poisson')

overall_mod

summary(overall_mod)

overall_tab <- cbind(exp(coef(overall_mod)), exp(confint(overall_mod)))
write.csv(overall_tab, file.path(results_dir, 'los_model_overall_inpatient.csv'))

overall_tab

### count models by wave, comparing poisson to negative binomial regression

In [ ]:
poisson_los_pop <- function(df, which_population, predictors) {
    
  data <- 
    df %>%
    dplyr::filter(
      population == which_population,
      !is.na(race)
    ) %>%
    dplyr::select(los, dplyr::all_of({{ predictors }}))
  
  mod <- glm(los ~ ., data = data, family = 'poisson')

  mod_summary <- summary(mod)

  mod_coef <- cbind(exp(coef(mod)), exp(confint(mod)))
  mod_coef_xtable <- xtable::xtable(mod_coef)

  out <- 
    list(
      mod = mod, 
      summary = mod_summary, 
      coef = mod_coef, 
      coef_xtable = mod_coef_xtable,
      aic = AIC(mod)
    )

  out
}

In [ ]:
negbin_los_pop <- function(df, which_population, predictors) {
    
  data <- 
    df %>%
    dplyr::filter(
      population == which_population,
      !is.na(race)
    ) %>%
    dplyr::select(los, dplyr::all_of({{ predictors }}))
  
  mod <- MASS::glm.nb(los ~ ., data = data)

  mod_summary <- summary(mod)

  mod_coef <- cbind(exp(coef(mod)), exp(confint(mod)))
  mod_coef_xtable <- xtable::xtable(mod_coef)

  out <- 
    list(
      mod = mod, 
      summary = mod_summary, 
      coef = mod_coef, 
      coef_xtable = mod_coef_xtable,
      aic = AIC(mod)
    )

  out
}

In [ ]:
wild_los_pois_mod <- poisson_los_pop(pois_inpatient, 'wild', pred_pois)
wild_los_negbin_mod <- negbin_los_pop(pois_inpatient, 'wild', pred_pois)

print(paste0('poisson aic: ', round(wild_los_pois_mod$aic, 2)))
print(paste0('neg bin: ', round(wild_los_negbin_mod$aic, 2)))

print(wild_los_negbin_mod)

In [ ]:
dec2020_los_pois_mod <- poisson_los_pop(pois_inpatient, 'dec2020', pred_pois)
dec2020_los_negbin_mod <- negbin_los_pop(pois_inpatient, 'dec2020', pred_pois)

print(paste0('poisson aic: ', round(dec2020_los_pois_mod$aic, 2)))
print(paste0('neg bin: ', round(dec2020_los_negbin_mod$aic, 2)))

print(dec2020_los_negbin_mod)

In [ ]:
delta_los_pois_mod <- poisson_los_pop(pois_inpatient, 'delta', pred_pois)
delta_los_negbin_mod <- negbin_los_pop(pois_inpatient, 'delta', pred_pois)

print(paste0('poisson aic: ', round(delta_los_pois_mod$aic, 2)))
print(paste0('neg bin: ', round(delta_los_negbin_mod$aic, 2)))

print(delta_los_negbin_mod)

In [ ]:
omicron_los_pois_mod <- poisson_los_pop(pois_inpatient, 'omicron', pred_pois)
omicron_los_negbin_mod <- negbin_los_pop(pois_inpatient, 'omicron', pred_pois)

print(paste0('poisson aic: ', round(omicron_los_pois_mod$aic, 2)))
print(paste0('neg bin: ', round(omicron_los_negbin_mod$aic, 2)))

print(omicron_los_negbin_mod)

In [ ]:
coef_list <- 
  list(
    wild = wild_los_negbin_mod$coef,
    dec2020 = dec2020_los_negbin_mod$coef, 
    delta = delta_los_negbin_mod$coef, 
    omicron = omicron_los_negbin_mod$coef
  )

coef_nb_df <- 
  Map(\(x) tibble::rownames_to_column(as.data.frame(x)), coef_list) |> 
  bind_rows(.id = 'population') |>
  janitor::clean_names()

black_nb_rr <- 
  coef_nb_df |> 
  dplyr::filter(rowname %in% c('race.factorBlack or African American')) |>
  dplyr::mutate(
    population = stringr::str_to_title(population),
    population = factor(population, levels = c("Wild", "Dec2020", "Delta", "Omicron"))
  )

In [ ]:
black_nb_rr_tab <- 
  black_nb_rr |> 
  dplyr::mutate(
    rowname = 'Black or African American',
    across(v1:x97_5_percent, ~ round(.x, 4))
  ) |>
  dplyr::rename(
    Estimate = v1,
    `2.5%` = x2_5_percent,
    `97.5%` = x97_5_percent
  ) |>
  dplyr::rename(
    Wave = population,
    Population = rowname
  )

print.xtable(
  xtable::xtable(black_nb_rr_tab, make.names = FALSE),
  type = 'latex', 
  file = file.path(results_dir, 'adjusted_NB_rr_inpatient.tex'), 
  include.rownames = FALSE
)

write.csv(
  black_nb_rr_tab, 
  file = file.path(results_dir, 'adjusted_NB_rr_inpatient.csv')
)

In [ ]:
black_nb_rr_graph <- 
  ggplot(black_nb_rr, aes(x = population, y = v1)) +
  geom_hline(yintercept = 1, linetype = 'dashed') +
  geom_pointrange(mapping = aes(ymin = x2_5_percent, ymax = x97_5_percent)) +
  labs(x = '', y = stringr::str_wrap('NB regression IRR length of stay', 35)) + 
  theme_bw() +
  theme(axis.text.x = element_text(colour = 'black', angle = 45, vjust = 0.5, hjust = 1))

black_nb_rr_graph_short <- black_nb_rr_graph
black_nb_rr_graph <- black_nb_rr_graph + ylim(0, NA)



ggsave(
  plot = black_nb_rr_graph, 
  filename = here::here('results', 'adjusted_NB_regressions_inpatient.pdf'), 
  width = 4, 
  height = 4
)
ggsave(
  plot = black_nb_rr_graph, 
  filename = here::here('results', 'adjusted_NB_regressions_inpatient.png'), 
  width = 4, 
  height = 4
)
ggsave(
  plot = black_nb_rr_graph, 
  filename = here::here('results', 'adjusted_NB_regressions_inpatient.svg'), 
  width = 4, 
  height = 4
)

ggsave(
  plot = black_nb_rr_graph_short, 
  filename = here::here('results', 'adjusted_NB_regressions_short_y_axis_inpatient.pdf'), 
  width = 4, 
  height = 4
)
ggsave(
  plot = black_nb_rr_graph_short, 
  filename = here::here('results', 'adjusted_NB_regressions_short_y_axis_inpatient.png'), 
  width = 4, 
  height = 4
)
ggsave(
  plot = black_nb_rr_graph_short, 
  filename = here::here('results', 'adjusted_NB_regressions_short_y_axis_inpatient.svg'), 
  width = 4, 
  height = 4
)

### reg result composite graph

In [ ]:
patchwork <- 
  ( black_log_or_graph + black_log_or_graph_short ) /
  ( black_nb_rr_graph + black_nb_rr_graph_short )

patchwork <- 
  patchwork + plot_annotation(tag_levels = 'A')

patchwork

ggsave(
  plot = patchwork,
  filename = here::here('results', 'regression_results_composite_inpatient.png'), 
  width = 6, 
  height = 6
)
ggsave(
  plot = patchwork,
  filename = here::here('results', 'regression_results_composite_inpatient.svg'), 
  width = 6, 
  height = 6
)
ggsave(
  plot = patchwork,
  filename = here::here('results', 'regression_results_composite_inpatient.pdf'), 
  width = 6, 
  height = 6
)

In [ ]:
# ---- AIC comparison table

aic_tab <- 
  tibble(
    Population = c('Dec2020', 'Delta', 'Omicron'),
    AIC_poisson = c(dec2020_los_pois_mod$aic, delta_los_pois_mod$aic, omicron_los_pois_mod$aic),
    AIC_nb = c(dec2020_los_negbin_mod$aic, delta_los_negbin_mod$aic, omicron_los_negbin_mod$aic)
  ) |>
  dplyr::mutate(
    delta_aic = AIC_nb - AIC_poisson,
    across(c(AIC_poisson, AIC_nb, delta_aic), ~ round(.x, 2))
  ) |>
  dplyr::rename(`AIC Poisson` = AIC_poisson, `AIC NB` = AIC_nb, `Delta AIC` = delta_aic)

aic_tab

write.csv(aic_tab, file = file.path(results_dir, 'aic_table_inpatient.csv'))

print.xtable(
  xtable::xtable(aic_tab, make.names = FALSE),
  type = 'latex', 
  file = file.path(results_dir, 'aic_table_inpatient.tex'), 
  include.rownames = FALSE
)

In [ ]:
# ------ citations ------

In [ ]:
R.Version()
citation()

citation('dplyr')
citation('ggplot2')
citation('gtools')
citation('MASS')
citation('xtable')
citation('bit64')
citation('scales')